### Lehmann data prep
##### 22 Feb 2024, Tin-Yu Hui
This notebook converts the original Lehmann et al. (2017) dataset, LehmannSNPdryad.csv, into our working csv for ABC. 

This notebook does not perform any filtering, it simply converts the input dataset into a matrix, with each row being a mosquito. The first four columns are the metadata: PoD (population ID), MosID (unique mosquito ID), Sex, CollDate (date of collection). The subsequent columns are the genotypes, which are coded as (0, 1, 2, NA). 0 and 2 are the homozygotes, 1 is the heterozygote, NA is missing. 

There are 505 mosquitoes and 724 columns (4 metadata columns + 720 loci). 

Downstream filtering (e.g. maf) is performed in the ABC notebook, as it needs to be done on a case-by-case basis. 

In [1]:
# READ THE RAW LEHMANN DATASET (JUST SAVE AS csv FROM THE ORIGINAL xlsx FILE)
dat<-read.csv('LehmannSNPdryad.csv', header=T, stringsAsFactors=F)
dim(dat)
names(dat)
head(dat)

[1] 372690      6

[1] "PoD"      "MosID"    "Sex"      "CollDate" "Locu"     "Geno"

PoD,MosID,Sex,CollDate,Locu,Geno
Ba10Se,Bg001BG1,F,13-Sep-10,2L10328509,G G
Ba10Se,Bg005BG5,F,13-Sep-10,2L10328509,A A
Ba10Se,Bg007BG7,F,13-Sep-10,2L10328509,0 0
Ba10Se,Bg008BG8,F,13-Sep-10,2L10328509,A G
Ba10Se,Bg009BG9,F,13-Sep-10,2L10328509,G G
Ba10Se,Bg010B10,F,13-Sep-10,2L10328509,G G


In [2]:
# FIND UNIQUE MOSQUITOS (AND THE MANDATORY COLUMNS)
temp<-!duplicated(dat[,1:4])
dat1<-dat[temp,1:4]
dim(dat1)
dat1<-data.frame(dat1)
names(dat1)
rm(temp)
head(dat1)

[1] 505   4

[1] "PoD"      "MosID"    "Sex"      "CollDate"

PoD,MosID,Sex,CollDate
Ba10Se,Bg001BG1,F,13-Sep-10
Ba10Se,Bg005BG5,F,13-Sep-10
Ba10Se,Bg007BG7,F,13-Sep-10
Ba10Se,Bg008BG8,F,13-Sep-10
Ba10Se,Bg009BG9,F,13-Sep-10
Ba10Se,Bg010B10,F,13-Sep-10


In [3]:
# FIND UNIQUE LOCI
unique_Locu<-unique(dat$Locu)
length(unique_Locu)

[1] 738

In [4]:
# CREATE A GENOTYPE DATA FRAME
# WITH 505 ROWS AND 738 COLUMNS
dat2<-data.frame(matrix(nr=nrow(dat1), nc=length(unique_Locu)))
names(dat2)<-unique_Locu
names(dat2)[1:5] # SHOW THE FIRST FEW COLUMN NAMES

[1] "2L10328509" "2L11053982" "2L11709879" "2L11960688" "2L12178159"

In [5]:
# FOR EACH LOCUS
for (i in 1:length(unique_Locu))
{
    # GET ALL INDIVIDUALS WITH THAT LOCUS, SHOULD BE 505 ROWS
    temp<-dat[dat$Locu==unique_Locu[i], c(2,6)]
    # THE ORDER IS NOT GUARANTEED SO NEED TO MERGE
    temp1<-merge(dat1[,2], temp, by=1, all.x=T, sort=F)
    # EXTRACT GENOTYPE
    temp1$Geno<-gsub(' ', '', temp1$Geno)
    g1<-substr(temp1$Geno, 1, 1)
    g2<-substr(temp1$Geno, 2, 2)
    # WE KNOW 0 MEANS MISSING VALUE SO TRANSFORM IT TO NA FIRST
    g1[g1=='0']<-NA
    g2[g2=='0']<-NA
    # FIND UNIQUE ALLELES?
    temp2<-sort(unique(c(g1, g2)))
    new_g1<-rep(NA, length(g1))
    new_g2<-rep(NA, length(g2))
    # ASSUME BIALLELIC, AND RECODE THE GENOTYPE. IF IT BELONGS TO temp2[1] THEN IT IS A 1, OTHERWISE 0
    new_g1<-(g1==temp2[1])
    new_g2<-(g2==temp2[1])
    # ADD THEM UP SUCH THAT THE POSSIBLE OUTCOMES ARE (0, 1, 2) AND NA
    dat2[,i]<-new_g1+new_g2
}

In [6]:
# SEE HOW THE FIRST AND THE LAST LOCUS LOOK LIKE
dat2[,1]
dat2[,738]

[1]  0  2 NA  1  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  2  2  1  0
 [26]  0  1  2  0 NA NA  1  0 NA  1  1  1  0  2  0  1  0 NA  2  1  0  1  1  0  2
 [51]  2  0  2  1  0  0  1 NA NA  1  0  0  1  2  1  0  1  1  0  0  0  0  0  1  2
 [76]  1  1  0  1  0  1  0  0  0  0  0  0  0  1  0  0  0  1  0  1  0  0  1  1  1
[101]  0  0  2  1  1 NA  1  1  0  2 NA  0  0  0  2  1  1  0  1 NA  1  0  0  1  1
[126]  0  1  0  1  1  2  0 NA  0  0  0 NA NA  1  0  1  0  0 NA  1  0  0 NA NA  0
[151]  1  1  1  1  0  1  1  0  0  0  1  0  1  0  0  0  0  2  1  2 NA  1  0  1  1
[176] NA  0  1  0  0  1  0  0  0  1  0  0  1  0  0  0 NA  0  0  1  0  1  0  1  2
[201]  1  0  1  0  1  0  0  1 NA NA NA  1 NA  1  1 NA NA NA  0  0  0 NA NA NA  0
[226] NA  1  0  1 NA NA NA NA  0  2  2  2 NA  1  1  1  0 NA  1 NA NA  0 NA NA  1
[251]  0  0  1 NA  0 NA  1  1 NA  1  0  0  1  1  1  0  1  0  0  0  0  1  0  1  0
[276]  0  1  0  0  0  1  0  0  0  2  0  0  1  0  1  1  1  0  2  0  1  1  2  2  1
[301]  1  0  1  1  2  1  1  2  1  2  0  1  1  2  0  1  2  0  1 NA  1  0  1  0  0
[326]  0  1  0  1  0  0  0  1  1  0  1  2  1  1  1  0  0 NA  1  0  1 NA  2  0  1
[351]  1  0  0 NA NA  0  0  0  0  1 NA  1  1  1  1  0  1  0  0  1  2  0  0  1  0
[376]  0  0  0  1  1  0  2  0  0  0  0  0  0 NA  0  1 NA  1  0 NA  1  1  2  0  1
[401]  0  0  0  0  0  0  0 NA  0  2  0  0  1  1  0  0  1  0  1  0  1  1  0  0  1
[426]  1  0  0  1  0  0  0  0  0  1  0  0  0  0  0  0  1  0  1  0  1  0  0  0  0
[451]  1  0  0  0  0  1  2  0  0  0  2  0 NA  1 NA  2 NA  0  0  0  1  1  0  1  0
[476]  0  0  1  1  0 NA  0  1  0  1  0  1  1  0  1  1  2  0  1  0  1  0  2  1  0
[501]  0  1 NA  0  2

[1]  1  0  2  1  1  0 NA  1  0  2  0  0  0  1  1  2  2  2  0  1  0  0  2  0  0
 [26]  2  2  2  1  0  1 NA  2  1  0  1  1  1  1  0 NA NA  2  1  2  2  1  1  1  0
 [51]  0  2  1  1  2 NA  2  1  1  1  1  2  0  1  0  1  2  1  0  2  2  2  1  1 NA
 [76]  0  0  1  2  1  2  1  0  0  2  0  0  2  2  2  1  1  1  0  2  1  1  2  2  0
[101]  2 NA  0  1  1  2  1  0  1  0  1  1  0  1  1  1  1  1  1 NA  0  1  1  1  1
[126]  1  2  1  0  2  0  0  1  2  1  1 NA  2 NA  0  1  1  2  0  1  2  2  0  0  1
[151]  2  1  1  1  0  1  1  1  0  2  0  0  2  0  1  0  2  1  0  2  1  1  1  1  1
[176]  0 NA  0  1  2 NA  0  2  0  1  1  0 NA NA NA  1  0  2  1  0  2  1  0  1  1
[201]  0  1  0 NA  1  2 NA  0  1  1  1  1  1  1 NA NA  1  1 NA  1 NA  1  1  1  1
[226]  1  0  1  1  1  1  1  1  2  2  1  0  1 NA NA  1  0  2  1  1  1  0  0 NA  2
[251]  1  1  0  1  1 NA  2  2 NA  1  2  0  1  1  0 NA  1  1  0  2  0 NA  1  0  1
[276]  1  0  2  1  0  1 NA  1  0  1  2  1  1  1 NA  0  2  2  1  2  2  0  2  1  0
[301]  1  2  0  2  0  1  0  1  0  1  1  1  1  0  0  0  1  0  2  1  2  1  1  1  1
[326]  0  1  1 NA  0  0  1 NA  1  0  2  0 NA  2  1  1  2  1  1  1 NA  0  0  1  0
[351]  0  0  1  2  1  2 NA  0  0  0  1  2  0  0  2  1  0  0  1  1 NA  0 NA  1  1
[376] NA  0  1  1  0  2  2  1  1  1  1  1  2  2  1  0  1  0  0  1  1  2  0  0  1
[401]  1  2  1  0  0  2  2  2  0  2  2  2  1 NA  0  0  1  1  2  1  1  1  0  0  0
[426]  0  1  2  1  1  1  2  0  2  2  1  0  0  2  0  2  2  1  1  2  2  1  0  1  0
[451]  2  0  2  0  1  1  1  1  2  1  1  0  1  0 NA  2  0  2  2  0  0  1  1  1  0
[476]  1  1  2  1  2  2  2  2  0  1 NA  1  2  0  1  1  1  2  1  1  0  1  0  1  2
[501]  1  1  2  1  0

In [7]:
# cbind THE TWO DATA FRAMES
dat3<-cbind(dat1, dat2)
dim(dat3)
head(dat3)

[1] 505 742

PoD,MosID,Sex,CollDate,2L10328509,2L11053982,2L11709879,2L11960688,2L12178159,2L12407388,...,3R9861447,3R9900077,3R9911823,3R9911990,3R9912044,3R9912122,3R9921698,3R9923311,3R9926264,3R9984283
Ba10Se,Bg001BG1,F,13-Sep-10,0,2,1,2,0,1,...,0,2,0,1,1,0,0,0,1,1
Ba10Se,Bg005BG5,F,13-Sep-10,2,2,0,1,0,0,...,0,2,1,0,0,0,0,0,0,0
Ba10Se,Bg007BG7,F,13-Sep-10,NA,2,0,1,0,0,...,0,2,1,2,1,1,1,0,0,2
Ba10Se,Bg008BG8,F,13-Sep-10,1,2,0,1,1,1,...,1,2,0,2,2,2,0,0,0,1
Ba10Se,Bg009BG9,F,13-Sep-10,0,1,1,1,1,0,...,0,2,0,2,1,1,0,0,0,1
Ba10Se,Bg010B10,F,13-Sep-10,0,2,0,1,1,1,...,0,2,0,2,0,0,0,0,0,0


In [8]:
# FINALLY SAVE IT AS csv
write.csv(dat3, 'Lehmann_coded_tinyu.csv', row.names=F)